In [1]:
import os
import string
from string import digits
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
import pandas as pd
import numpy as np
import string
from string import digits
import re
from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split
from tensorflow.keras.layers import Input, LSTM, Embedding, Dense
from tensorflow.keras.models import Model
import tensorflow as tf

In [2]:
data=pd.read_csv("../input/hindienglish-corpora/Hindi_English_Truncated_Corpus.csv",encoding='utf-8',names=['source','lang','trans','len_lang_sent','len_trans_sent'])

In [3]:
data=data.iloc[1:,:]
data.head()

source                                               lang  \
1        ted  politicians do not have permission to do what ...   
2        ted         I'd like to tell you about one such child,   
3  indic2012  This percentage is even greater than the perce...   
4        ted  what we really mean is that they're bad at not...   
5  indic2012  .The ending portion of these Vedas is called U...   

                                               trans  len_lang_sent  \
1  राजनीतिज्ञों के पास जो कार्य करना चाहिए, वह कर...            NaN   
2  मई आपको ऐसे ही एक बच्चे के बारे में बताना चाहू...            NaN   
3   यह प्रतिशत भारत में हिन्दुओं प्रतिशत से अधिक है।            NaN   
4     हम ये नहीं कहना चाहते कि वो ध्यान नहीं दे पाते            NaN   
5        इन्हीं वेदों का अंतिम भाग उपनिषद कहलाता है।            NaN   

   len_trans_sent  
1             NaN  
2             NaN  
3             NaN  
4             NaN  
5             NaN

In [4]:
data=data[data.source=='ted']

In [5]:
pd.isnull(data).sum()

source                0
lang                  0
trans                 0
len_lang_sent     39881
len_trans_sent    39881
dtype: int64

In [6]:
data=data[~pd.isnull(data.trans)]
data.shape

(39881, 5)

In [7]:
data.drop_duplicates(inplace=True)
data.drop(data[data.lang.str.contains(r'[0-9]')].index, inplace = True)
data.shape

(37036, 5)

In [8]:
spchrs=set(string.punctuation)
lowerCase=lambda x:x.lower()
quotes=lambda x:re.sub("'",'',x)
specialCh=lambda x:''.join(ch for ch in x if ch not in spchrs)
rmDigits=lambda x:x.translate(str.maketrans('','',digits))
spaces=lambda x:x.strip()
unwanted=lambda x:re.sub(" +"," ",x)
startEnd=lambda x:'START_ '+x+' _END'
data.lang=data.lang.apply(quotes)
data.lang=data.lang.apply(specialCh)
data.lang=data.lang.apply(rmDigits)
data.lang=data.lang.apply(spaces)
data.lang=data.lang.apply(unwanted)
data.lang=data.lang.apply(lowerCase)
data.trans=data.trans.apply(quotes)
data.trans=data.trans.apply(specialCh)
data.trans=data.trans.apply(rmDigits)
data.trans=data.trans.apply(spaces)
data.trans=data.trans.apply(unwanted)
data.trans=data.trans.apply(lowerCase)
data.trans=data.trans.apply(startEnd)

In [9]:
data.head()

source                                               lang  \
1     ted  politicians do not have permission to do what ...   
2     ted           id like to tell you about one such child   
4     ted  what we really mean is that theyre bad at not ...   
8     ted     and who are we to say even that they are wrong   
14    ted                   so there is some sort of justice   

                                                trans  len_lang_sent  \
1   START_ राजनीतिज्ञों के पास जो कार्य करना चाहिए...            NaN   
2   START_ मई आपको ऐसे ही एक बच्चे के बारे में बता...            NaN   
4   START_ हम ये नहीं कहना चाहते कि वो ध्यान नहीं ...            NaN   
8   START_ और हम होते कौन हैं यह कहने भी वाले कि व...            NaN   
14                       START_ तो वहाँ न्याय है _END            NaN   

    len_trans_sent  
1              NaN  
2              NaN  
4              NaN  
8              NaN  
14             NaN

In [10]:
all_lang_words=set()
for line in data.lang:
    for word in line.split():
        if word not in all_lang_words:
            all_lang_words.add(word)

all_trans_words=set()
for line in data.trans:
    for word in line.split():
        if word not in all_trans_words:
            all_trans_words.add(word)
print(len(all_lang_words),len(all_trans_words))

16992 21745


In [11]:
data.head()

source                                               lang  \
1     ted  politicians do not have permission to do what ...   
2     ted           id like to tell you about one such child   
4     ted  what we really mean is that theyre bad at not ...   
8     ted     and who are we to say even that they are wrong   
14    ted                   so there is some sort of justice   

                                                trans  len_lang_sent  \
1   START_ राजनीतिज्ञों के पास जो कार्य करना चाहिए...            NaN   
2   START_ मई आपको ऐसे ही एक बच्चे के बारे में बता...            NaN   
4   START_ हम ये नहीं कहना चाहते कि वो ध्यान नहीं ...            NaN   
8   START_ और हम होते कौन हैं यह कहने भी वाले कि व...            NaN   
14                       START_ तो वहाँ न्याय है _END            NaN   

    len_trans_sent  
1              NaN  
2              NaN  
4              NaN  
8              NaN  
14             NaN

In [12]:
data.len_lang_sent=data.lang.apply(lambda x:len(x.split(" ")))
data.len_trans_sent=data.trans.apply(lambda x:len(x.split(" ")))

In [13]:
data.head()

source                                               lang  \
1     ted  politicians do not have permission to do what ...   
2     ted           id like to tell you about one such child   
4     ted  what we really mean is that theyre bad at not ...   
8     ted     and who are we to say even that they are wrong   
14    ted                   so there is some sort of justice   

                                                trans  len_lang_sent  \
1   START_ राजनीतिज्ञों के पास जो कार्य करना चाहिए...             12   
2   START_ मई आपको ऐसे ही एक बच्चे के बारे में बता...              9   
4   START_ हम ये नहीं कहना चाहते कि वो ध्यान नहीं ...             12   
8   START_ और हम होते कौन हैं यह कहने भी वाले कि व...             11   
14                       START_ तो वहाँ न्याय है _END              7   

    len_trans_sent  
1               15  
2               13  
4               13  
8               15  
14               6

In [14]:
data=data[data.len_lang_sent<=20]
data=data[data.len_trans_sent<=20]
data.shape

(36721, 5)

In [15]:
data=data.sample(n=35000,random_state=42)

In [16]:
print("maximum length of lang Sentence ",max(data.len_lang_sent))
print("maximum length of trans Sentence ",max(data.len_trans_sent))

maximum length of lang Sentence  20
maximum length of trans Sentence  20


In [17]:
max_length_src=max(data.len_lang_sent)
max_length_tar=max(data.len_trans_sent)
input_words = sorted(list(all_lang_words))
target_words = sorted(list(all_trans_words))
num_encoder_tokens = len(all_lang_words)
num_decoder_tokens = len(all_trans_words)
num_encoder_tokens, num_decoder_tokens

(16992, 21745)

In [18]:
num_decoder_tokens+=1

In [19]:
input_token_index = dict([(word, i+1) for i, word in enumerate(input_words)])
target_token_index = dict([(word, i+1) for i, word in enumerate(target_words)])
reverse_input_char_index = dict((i, word) for word, i in input_token_index.items())
reverse_target_char_index = dict((i, word) for word, i in target_token_index.items())


In [20]:
data = shuffle(data)
data.head(10)

source                                               lang  \
48654     ted  i mean the citys the best gallery i could imagine   
74939     ted            by a really convincing way of saying it   
74255     ted                          associated with east asia   
85863     ted   you express yourself because you have compassion   
70261     ted                               start all over again   
33403     ted                       youre sitting there in class   
60714     ted        the concept of physical health is undefined   
13799     ted  youll see that most of the languages spoken in...   
57189     ted                                harvest palms smoke   
104088    ted                      of solarelectrifying villages   

                                                    trans  len_lang_sent  \
48654   START_ मेरा मतलब है शहर मेरी निगाह में सबसे बे...             10   
74939                   START_ से कहना बहुत जरूरी है _END              8   
74255       START_ जिसे पूर्व एशिया में देखा गया है। _END              4   
85863   START_ आप खुद को अभिवयक्त करते हो क्यूंकि आप म...              7   
70261                           START_ एक नयी शुरुआत _END              4   
33403              START_ तुम वहाँ कक्षा में बैठे हो _END              5   
60714         START_ आरोग्य की संकल्पना अपरिभाषित है _END              7   
13799   START_ तो आप देखेंगे कि उत्तर भारत में बोली जा...             11   
57189          START_ हथेलियों की फसल काट लो धुँआ है _END              3   
104088    START_ गाँवों को सौरऊर्जा देने के तरीके से _END              3   

        len_trans_sent  
48654               13  
74939                7  
74255                9  
85863               13  
70261                5  
33403                8  
60714                7  
13799               14  
57189                9  
104088               9

In [21]:
X, y = data.lang, data.trans
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2,random_state=42)
X_train.shape, X_test.shape

((28000,), (7000,))

In [22]:
X_train.to_pickle('X_train.pkl')
X_test.to_pickle('X_test.pkl')

In [23]:
def generate_batch(X = X_train, y = y_train, batch_size = 128):
    ''' Generate a batch of data '''
    while True:
        for j in range(0, len(X), batch_size):
            encoder_input_data = np.zeros((batch_size, max_length_src),dtype='float32')
            decoder_input_data = np.zeros((batch_size, max_length_tar),dtype='float32')
            decoder_target_data = np.zeros((batch_size, max_length_tar, num_decoder_tokens),dtype='float32')
            for i, (input_text, target_text) in enumerate(zip(X[j:j+batch_size], y[j:j+batch_size])):
                for t, word in enumerate(input_text.split()):
                    encoder_input_data[i, t] = input_token_index[word]
                for t, word in enumerate(target_text.split()):
                    if t<len(target_text.split())-1:
                        decoder_input_data[i, t] = target_token_index[word] 
                    if t>0:
                        decoder_target_data[i, t - 1, target_token_index[word]] = 1.
            yield([encoder_input_data, decoder_input_data], decoder_target_data)

In [24]:
latent_dim=50
encoder_inputs = Input(shape=(None,))
enc_emb =  Embedding(num_encoder_tokens, latent_dim, mask_zero = True)(encoder_inputs)
encoder_lstm = LSTM(latent_dim, return_state=True)
encoder_outputs, state_h, state_c = encoder_lstm(enc_emb)
encoder_states = [state_h, state_c]

2022-01-09 21:39:43.411829: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-01-09 21:39:43.549724: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-01-09 21:39:43.550769: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-01-09 21:39:43.552395: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags

In [25]:
decoder_inputs = Input(shape=(None,))
dec_emb_layer = Embedding(num_decoder_tokens, latent_dim, mask_zero = True)
dec_emb = dec_emb_layer(decoder_inputs)
decoder_lstm = LSTM(latent_dim, return_sequences=True, return_state=True)
decoder_outputs, _, _ = decoder_lstm(dec_emb,
                                     initial_state=encoder_states)
decoder_dense = Dense(num_decoder_tokens, activation='softmax')
decoder_outputs = decoder_dense(decoder_outputs)
model = Model([encoder_inputs, decoder_inputs], decoder_outputs)

In [26]:
model.compile(optimizer='rmsprop', loss='categorical_crossentropy',metrics=['accuracy'])
model.summary()

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, None)]       0                                            
__________________________________________________________________________________________________
input_2 (InputLayer)            [(None, None)]       0                                            
__________________________________________________________________________________________________
embedding (Embedding)           (None, None, 50)     849600      input_1[0][0]                    
__________________________________________________________________________________________________
embedding_1 (Embedding)         (None, None, 50)     1087300     input_2[0][0]                    
______________________________________________________________________________________________

In [27]:
train_samples = len(X_train)
val_samples = len(X_test)
batch_size = 128
epochs =100

In [28]:
model.fit(generate_batch(X_train, y_train, batch_size = batch_size),
          steps_per_epoch = train_samples//batch_size,
          epochs=epochs,
          shuffle=True,
          validation_data = generate_batch(X_test, y_test, batch_size = batch_size),
          validation_steps = val_samples//batch_size)

2022-01-09 21:39:50.209717: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:185] None of the MLIR Optimization Passes are enabled (registered 2)


Epoch 1/100


2022-01-09 21:40:01.371008: I tensorflow/stream_executor/cuda/cuda_dnn.cc:369] Loaded cuDNN version 8005


218/218 [==============================] - 132s 545ms/step - loss: 3.2495 - accuracy: 0.1060 - val_loss: 2.9631 - val_accuracy: 0.1078
Epoch 2/100
218/218 [==============================] - 135s 618ms/step - loss: 2.9180 - accuracy: 0.1270 - val_loss: 2.8773 - val_accuracy: 0.1363
Epoch 3/100
218/218 [==============================] - 116s 535ms/step - loss: 2.8365 - accuracy: 0.1401 - val_loss: 2.8243 - val_accuracy: 0.1465
Epoch 4/100
218/218 [==============================] - 117s 537ms/step - loss: 2.7588 - accuracy: 0.1518 - val_loss: 2.7541 - val_accuracy: 0.1592
Epoch 5/100
218/218 [==============================] - 117s 538ms/step - loss: 2.6869 - accuracy: 0.1679 - val_loss: 2.6967 - val_accuracy: 0.1771
Epoch 6/100
218/218 [==============================] - 118s 540ms/step - loss: 2.6234 - accuracy: 0.1836 - val_loss: 2.6508 - val_accuracy: 0.1873
Epoch 7/100
218/218 [==============================] - 117s 534ms/step - loss: 2.5706 - accuracy: 0.1942 - val_loss: 2.6178 - val_

In [29]:
model.save_weights('eng_to_hindi.h5')

In [30]:
encoder_model = Model(encoder_inputs, encoder_states)

decoder_state_input_h = Input(shape=(latent_dim,))
decoder_state_input_c = Input(shape=(latent_dim,))
decoder_states_inputs = [decoder_state_input_h, decoder_state_input_c]

dec_emb2= dec_emb_layer(decoder_inputs) 

decoder_outputs2, state_h2, state_c2 = decoder_lstm(dec_emb2, initial_state=decoder_states_inputs)
decoder_states2 = [state_h2, state_c2]
decoder_outputs2 = decoder_dense(decoder_outputs2)
decoder_model = Model(
    [decoder_inputs] + decoder_states_inputs,
    [decoder_outputs2] + decoder_states2)


In [31]:
def decode_sequence(input_seq):
    states_value = encoder_model.predict(input_seq)
    target_seq = np.zeros((1,1))
    target_seq[0, 0] = target_token_index['START_']
    stop_condition = False
    decoded_sentence = ''
    while not stop_condition:
        output_tokens, h, c = decoder_model.predict([target_seq] + states_value)
        sampled_token_index = np.argmax(output_tokens[0, -1, :])
        sampled_char = reverse_target_char_index[sampled_token_index]
        decoded_sentence += ' '+sampled_char
        if (sampled_char == '_END' or
           len(decoded_sentence) > 50):
            stop_condition = True
        target_seq = np.zeros((1,1))
        target_seq[0, 0] = sampled_token_index
        states_value = [h, c]

    return decoded_sentence



In [32]:
train_gen = generate_batch(X_train, y_train, batch_size = 1)
k=-1


In [33]:
k+=1
(input_seq, actual_output), _ = next(train_gen)
decoded_sentence = decode_sequence(input_seq)
print('Input English sentence:', X_train[k:k+1].values[0])
print('Actual Hindi Translation:', y_train[k:k+1].values[0][6:-4])
print('Predicted Hindi Translation:', decoded_sentence[:-4])

Input English sentence: to print a childs drawing of micky mouse
Actual Hindi Translation:  कि मिकी माउस का चित्र जो एक बच्चे ने बनाया है 
Predicted Hindi Translation:  कि एक बड़ी तरह की तुलना में और 


In [34]:
k+=1
(input_seq, actual_output), _ = next(train_gen)
decoded_sentence = decode_sequence(input_seq)
print('Input English sentence:', X_train[k:k+1].values[0])
print('Actual Hindi Translation:', y_train[k:k+1].values[0][6:-4])
print('Predicted Hindi Translation:', decoded_sentence[:-4])

Input English sentence: and paste them
Actual Hindi Translation:  और उन्हें चिपकायें 
Predicted Hindi Translation:  और उन्हें पुष्परेणु करने की क्षमता नहीं थी 


In [35]:
k+=1
(input_seq, actual_output), _ = next(train_gen)
decoded_sentence = decode_sequence(input_seq)
print('Input English sentence:', X_train[k:k+1].values[0])
print('Actual Hindi Translation:', y_train[k:k+1].values[0][6:-4])
print('Predicted Hindi Translation:', decoded_sentence[:-4])

Input English sentence: by the end of march the lhc will be intact again
Actual Hindi Translation:  मार्च के अंत तक lhc फिर से संपूर्ण हो जाएगा। 
Predicted Hindi Translation:  तालिबान समय होकर फिर से विकसित करने वाले लोग 


In [36]:
k+=1
(input_seq, actual_output), _ = next(train_gen)
decoded_sentence = decode_sequence(input_seq)
print('Input English sentence:', X_train[k:k+1].values[0])
print('Actual Hindi Translation:', y_train[k:k+1].values[0][6:-4])
print('Predicted Hindi Translation:', decoded_sentence[:-4])



Input English sentence: where media has changed enough to qualify for the label revolution
Actual Hindi Translation:  जहाँ मीडिया का बदलाव इतना रहा है कि उसे क्रांति कहा जा सके 
Predicted Hindi Translation:  जहां मीडिया ऊर्जा को बेहतर होगी 
